In [24]:
import torch.nn as nn
from datasets import load_dataset
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

ImportError: cannot import name 'tags' from 'transformers' (c:\Users\maxim\OneDrive\proy__transformers\transformers_env\lib\site-packages\transformers\__init__.py)

In [ ]:
# Tengo que agregar el codigo de la carga del dataset para ejecutar esto

In [22]:
class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    # Aca esta cambiando la configuracion de RobertaPretraindeModel
    # Se puede ver que es lo que modifica comparando RobertaPretraindeModel.config_class()
    config_class = XLMRobertaConfig
    

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
    
    # Defino el forward pass
    def forward(self, inputs_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        # Uso el body del modelo para obtener las representaciones del encoder
        outputs = self.roberta(inputs_ids, attention_mask=attention_mask, token_type_ids=token_type_iids, **kwargs)

        # Aplico el classificador a la representacion de output del encoder
        sequence_output = self.dropout(outputs[0])
        # Aplico el classifier
        logits = self.classifier(sequence_output)
        # Calculo las perdidas
        loss = None
        if labels is not None:
            # Inicializo la funcion de perdida
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return TokenClassifierOutput(loss=loss, logits=logits,
                                         hidden_states=outputs.hidden_states,
                                         attentions=outputs.attentions)